In [9]:
import cvxpy as cp
import numpy as np
import networkx as nx



inputnodes = [0,6]
inputedges = [(0,5),(0,3),(3,4),(4,5),(4,6),(5,2),(3,1),(1,6),(6,2)]

C = {
    (0,5):1,
    (0,3):1,
    (3,4):1,
    (4,5):1,
    (4,6):1,
    (5,2):1,
    (3,1):1,
    (1,6):1,
    (6,2):1
}

non_member = [3,4,5,6]
source = 0
receivers = [1,2]


preG = nx.Graph()
preG.add_nodes_from(inputnodes)
preG.add_edges_from(inputedges)
directE = []
for n, nbrs in preG.adj.items():
    for nbr, eattr in nbrs.items():
        directE.append((n,nbr))


G = nx.DiGraph()
G.add_nodes_from(inputnodes)
G.add_edges_from(directE)

undirected_edges = set(inputedges)

vertices = np.arange(inputnodes[1]+1)
print(vertices)

edges = set(directE)
print(edges)


#----------------------------- finding all paths -------------------#


def find_all_paths(G, source, receivers):
    all_paths = {}
    for receiver in receivers:
        paths = list(nx.all_simple_paths(G, source=source, target=receiver))
        all_paths[receiver] = paths
        print(f"All paths from {source} to {receiver}: {paths}")
    return all_paths

# Calling the function
all_paths = find_all_paths(G, source, receivers)
print(all_paths)




#----------------------------- Cflow Function -------------------#


def cflow(source, receivers, non_member, undirected_edges, edges, C):
    f_star = cp.Variable()  # Target flow rate


    orientation_constraints = []
    flow_constraints = []
    equalrate_constraints = []

    c =  {a : cp.Variable() for a in edges}

    for a in edges:
        orientation_constraints += [c[a] >= 0]      #for every in 2E

    for a1 in undirected_edges:         # for every in E
        a2 = (a1[1], a1[0]) 
        orientation_constraints += [ c[a1] + c[a2] == C[a1]]


    fia = {(i, a): cp.Variable() for a in edges for i in receivers}

    for i in receivers: 
        for a in edges:
            flow_constraints += [fia[i,a] <= c[a]]
            flow_constraints += [fia[i,a] >= 0]


    for i in receivers:
        for j in non_member:
            incoming_edges = [(u, v) for u, v in edges if v == int(j)] 
            outcoming_edges = [(u, v) for u, v in edges if u == int(j)] 
            incoming_flow = sum(fia[i, x] for x in incoming_edges)
            outcoming_flow = sum(fia[i, y] for y in outcoming_edges)
            flow_constraints += [incoming_flow == outcoming_flow]

        incoming_flow_source = sum(fia[i, x] for x in edges if x[1] == source)  
        
        outcoming_flow_reciever = sum(fia[i, x] for x in edges if x[0] == int(i)) 
        incoming_flow_reciever = sum(fia[i, x] for x in edges if x[1] == int(i))  

        flow_constraints += [incoming_flow_source == 0]
        flow_constraints += [outcoming_flow_reciever == 0]
        equalrate_constraints += [f_star == incoming_flow_reciever]


    objective = cp.Maximize(f_star)    
    constraints = orientation_constraints + flow_constraints + equalrate_constraints
    problem = cp.Problem(objective, constraints)


    problem.solve()

    print("Optimal Target Flow Rate (f*):", f_star.value)

    optimal_flow = f_star.value
    flowia = {(i, a): round(float(fia[i,a].value), 6) for a in edges for i in receivers}

    return(optimal_flow, flowia)


optimal_flow, flowia = cflow(source, receivers, non_member, undirected_edges, edges, C)
print(flowia)

#-----------------------------  Splitting cflow Function ---------------------------#

# single matrix for one receiver
def create_edge_path_matrix(G, all_paths, receiver):
    edges = list(G.edges())
    paths = all_paths[receiver]

    matrix = np.zeros((len(edges), len(paths)), dtype=int)
    edge_index = {edge: idx for idx, edge in enumerate(edges)}
    
    for path_idx, path in enumerate(paths):
        for u, v in zip(path[:-1], path[1:]):  # Create edges from path
            if (u, v) in edge_index:
                matrix[edge_index[(u, v)], path_idx] = 1
    return matrix

# matrices for all receivers
matrices = {}
for receiver in receivers:
    matrices[receiver] = create_edge_path_matrix(G, all_paths, receiver)
    print(f"Matrix for receiver {receiver}:\n{matrices[receiver]}\n")





[0 1 2 3 4 5 6]
{(6, 2), (3, 4), (4, 3), (3, 1), (6, 1), (5, 4), (0, 3), (4, 6), (6, 4), (3, 0), (4, 5), (5, 0), (2, 6), (0, 5), (1, 6), (2, 5), (1, 3), (5, 2)}
All paths from 0 to 1: [[0, 5, 4, 3, 1], [0, 5, 4, 6, 1], [0, 5, 2, 6, 4, 3, 1], [0, 5, 2, 6, 1], [0, 3, 4, 5, 2, 6, 1], [0, 3, 4, 6, 1], [0, 3, 1]]
All paths from 0 to 2: [[0, 5, 4, 3, 1, 6, 2], [0, 5, 4, 6, 2], [0, 5, 2], [0, 3, 4, 5, 2], [0, 3, 4, 6, 2], [0, 3, 1, 6, 4, 5, 2], [0, 3, 1, 6, 2]]
{1: [[0, 5, 4, 3, 1], [0, 5, 4, 6, 1], [0, 5, 2, 6, 4, 3, 1], [0, 5, 2, 6, 1], [0, 3, 4, 5, 2, 6, 1], [0, 3, 4, 6, 1], [0, 3, 1]], 2: [[0, 5, 4, 3, 1, 6, 2], [0, 5, 4, 6, 2], [0, 5, 2], [0, 3, 4, 5, 2], [0, 3, 4, 6, 2], [0, 3, 1, 6, 4, 5, 2], [0, 3, 1, 6, 2]]}
Optimal Target Flow Rate (f*): 1.9999999999932954
{(1, (6, 2)): 0.0, (2, (6, 2)): 1.0, (1, (3, 4)): 0.0, (2, (3, 4)): 1.0, (1, (4, 3)): 0.0, (2, (4, 3)): 0.0, (1, (3, 1)): 1.0, (2, (3, 1)): 0.0, (1, (6, 1)): 1.0, (2, (6, 1)): 0.0, (1, (5, 4)): 1.0, (2, (5, 4)): 0.0, (1, (0, 3)): 